In [2]:
import json
import numpy as np
import pandas as pd
import requests

In [3]:
key = open('APIkeys.txt', 'r')

In [4]:
keys = list(key)

In [5]:
api_key = keys[0].split()[2]

In [37]:
url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Pete Wells&fq=news_desk:(Dining) AND type_of_material:(Review)&page={page}&api-key={api_key}'



In [38]:
r = requests.get(url)

In [39]:
json_data = r.json()

In [55]:
r.json()['response']['docs'][4]['web_url']

'https://www.nytimes.com/2012/02/22/dining/reviews/allswell-nyc-restaurant-review.html'

In [44]:
def query_nytimes(api_key, n_page, news_desk = 'Dining', news_type = 'Review', query = 'Pete Wells'):
    url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&fq=news_desk:{news_desk} AND type_of_material:{news_type}&page={n_page}&api-key={api_key}'
    r = requests.get(url)
    json_data = r.json()
    return r.json()['response']['docs']

In [46]:
# query_nytimes(api_key, 1)

In [ ]:
query constructor- **kwargs w/ all values set to None except query and api_key
base url = (base url)
if/elif/else
base url += news desk, query, section, date range, etc etc

In [95]:
nytimes_query(api_key, 'Pete Wells')

{'fault': {'faultstring': 'Failed to resolve API Key variable request.queryparam.api-key',
  'detail': {'errorcode': 'steps.oauth.v2.FailedToResolveAPIKey'}}}

In [93]:
def nytimes_query(api_key, query, 
                  glocations = None, headline = None, news_desk = None, 
                  organizations = None, persons = None, subject = None,
                  news_type = None, type_of_material = None, 
                  begin_date = None, end_date = None, n_page = 1):
    ''''''
    base_url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}'             
    filter_queries = {}
    dates = {}

    if glocations:
        filter_queries.update({'glocations': glocations})
    if headline:
        filter_queries.update({'headline': headline})
    if news_desk:
        filter_queries.update({'news_desk': news_desk})
    if organizations:
        filter_queries.update({'organizations': organizations})
    if persons:
        filter_queries.update({'persons': persons})
    if subject:
        filter_queries.update({'subject': subject})
    if news_type:
        filter_queries.update({'news_type': news_type})
    if type_of_material:
        filter_queries.update({'type_of_material': type_of_material})
    
    if begin_date:
        dates.update({'begin_date': begin_date})
    if end_date:
        dates.update({'end_date':end_date})

    if len(filter_queries) == 0 and len(dates) == 0:
        base_url += f'&api-key={api_key}'
        r = requests.get(base_url)
        json_data = r.json()
        return r.json()['response']['docs']
    
    elif len(filter_queries) == 1:
        base_url += f'&fq=({list(filter_queries.keys())[0]}):{list(filter_queries.values())[0]}'
        if len(dates) == 1:
            base_url += f'&{list(filter_queries.keys())[0]}={list(filter_queries.values())[0]}&'
        elif len(dates) == 2:
            base_url += '&'
            for i in dates.keys():
                base_url += f'{i}={dates[i]}&'

        base_url += f'api-key={api_key}'
        r = requests.get(base_url)
        json_data = r.json()
        return r.json()['response']['docs']

    elif  len(filter_queries) > 1:
        for i in filter_queries.keys():
            base_url += f'&fq=({i}):{filter_queries[i]} AND'
        base_url = base_url[:-4]
        if len(dates) == 1:
            base_url += f'&{list(filter_queries.keys())[0]}={list(filter_queries.values())[0]}&'
        elif len(dates) == 2:
            base_url += '&'
            for i in dates.keys():
                base_url += f'{i}={dates[i]}&'

        base_url += f'api-key={api_key}'
        r = requests.get(base_url)
        json_data = r.json()
        return r.json()['response']['docs']


    else:
        r = requests.get(base_url)
        json_data = r.json()
        return r.json()['response']['docs']


In [87]:
# nytimes_query(api_key, 'Pete Wells', news_desk = 'Dining', news_type = 'Review')

In [75]:
d = {'begin_date' : '20201201', 'end_date': '20210131'}

In [79]:
for i in d.keys():
    print(d[i])

20201201
20210131


In [80]:
strng = 'I want to buy a guava AND a banana AND a pear AND'

In [85]:
strng[:-4]

'I want to buy a guava AND a banana AND a pear'

In [96]:
f'&fq=({list(d.keys())[0]}):{list(d.values())[0]}'

'&fq=(begin_date):20201201'